In [12]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [ ]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

In [ ]:
# gera uma lista de tupla com as possibilidades dos codigos de serventias e um intervalo dos anos
servs_anos = list(itertools.product(list(serventias_jec.cod_serv),range(2013,2017)))

# gera a lista dos arquivos que deverão ser lidos
processos_serv_ano = [TJData.processo(sa[0],sa[1]) for sa in servs_anos]

# colunas que serão utilizadas no processo
usecols=['COD_PROC','COD_COMP','COD_SERV','ID_PROC','COD_ASSUNTO']

# filtra os arquivos inexistentes
#processos_serv_ano = TJData.try_read(processos_serv_ano,usecols=usecols)

In [ ]:
r'{}'.format(TJData.processo('(\d+)','(\d+)'))

In [ ]:
def in_serv_ano(file, file_regex, servs_anos):
    print(tuple(map(lambda x: int(x),re.findall(r'(\d+)',file))) in servs_anos)
    return tuple(map(lambda x: int(x),re.findall(r'(\d+)',file))) in servs_anos
    
    

def list_files_in_serventias_anos(file_regex, servs_anos):
    dirpath = re.search(r'(\w+/)', file_regex).group(0)
    fulldirpath = os.path.join('/tj_files',dirpath)
    onlyfiles = [
        f for f in os.listdir(fulldirpath)
        if os.path.isfile(os.path.join(fulldirpath, f))
        and in_serv_ano(f, file_regex ,servs_anos)
    ]
    return onlyfiles
list_files_in_serventias_anos(TJData.processo('(\d+)','(\d+)'),serventias_jec)

In [ ]:
servs_anos

in_db: 576431 original: 590392
out_db: 13961


In [ ]:
out_db_splitted = api.split_list(out_db, len(out_db)//1000)

In [8]:
len(out_db_splitted)

19

In [17]:
api.get('classesassuntos')

<Response [200]>